# Run IEDB API

In [3]:
import requests
import ast
import time
import json

## Run predictions

In [49]:
# Insert parameters here:

fasta = "input.fa"
output_file = "output.json"
alleles = "H2-Kb,H2-Db"
peptide_length_range = [9, 10]
method = "netmhcpan_el"

##########################

time_before_next_attempt = 15

sequences = open(fasta, 'r').read()

url = "https://api-nextgen-tools.iedb.org/api/v1/pipeline"

headers = {"accept": "application/json", 
           "Content-type": "application/json"}

payload = {
  "pipeline_title": "MHC prediction",
  "run_stage_range": [
    1,
    1
  ],
  "stages": [
    {
      "stage_number": 1,
      "tool_group": "mhci",
      "input_sequence_text": sequences,
      "input_parameters": {
        "alleles": alleles,
        "peptide_length_range": peptide_length_range,
        "predictors": [
          {
            "type": "binding",
            "method": method
          }
        ]
      }
    }
  ]
}

response = requests.post(url=url, json=payload, headers=headers)

if response.status_code == 200:
    print("Query succesfully submitted")
    result_link = ast.literal_eval(response.content.decode('utf-8'))["results_uri"]
    print(f"Getting results from {result_link}")
    
    results_status = "pending"
    while results_status != "done":
        results = requests.get(result_link)
        results = json.loads(results.content.decode('utf-8'))
        results_status = results["status"]
        print(f"Results are still pending, waiting {time_before_next_attempt}s before next attempt")
        time.sleep(time_before_next_attempt)
    
    print(f"Results completed. Saving at {output_file}")
    
    with open(output_file, "w") as fh:
        json.dump(results, fh)
        

else: 
    print(f"An error occurred. Status code is: {response.status_code}")

Query succesfully submitted
Getting results from https://api-nextgen-tools.iedb.org/api/v1/results/231a5d52-2d2d-46b9-93eb-4a58d3f721ca
Results are still pending, waiting 15s before next attempt
Results are still pending, waiting 15s before next attempt
Results completed. Saving at output.json


## Parse results

In [7]:
with open(output_file, "r") as fh:
    results_dic = json.load(fh)

In [13]:
print(results_dic['data']['results'][0])

{'type': 'peptide_table', 'table_data': [[1, 'ALPHIIDEV', 10, 18, 9, 'H2-Db', 10, 1.2, 'ALPHIIDEV', 'ALPHIIDEV', 0.06681, 1.2], [1, 'ALPHIIDEV', 10, 18, 9, 'H2-Kb', 10, 6.4, 'ALPHIIDEV', 'ALPHIIDEV', 0.009798, 6.4], [1, 'ALPHIIDEVI', 10, 19, 10, 'H2-Db', 34, 2.7, 'ALPHIIDEI', 'ALPHIIDEVI', 0.024745, 2.7], [1, 'ALPHIIDEVI', 10, 19, 10, 'H2-Kb', 34, 32.0, 'ALPHIIDEI', 'ALPHIIDEVI', 0.000444, 32.0], [1, 'DEVINIVII', 16, 24, 9, 'H2-Db', 16, 3.1, 'DEVINIVII', 'DEVINIVII', 0.020514, 3.1], [1, 'DEVINIVII', 16, 24, 9, 'H2-Kb', 16, 30.0, 'DEVINIVII', 'DEVINIVII', 0.00051, 30.0], [1, 'DEVINIVIIV', 16, 25, 10, 'H2-Db', 40, 23.0, 'DEVINIVIV', 'DEVINIVIIV', 0.000727, 23.0], [1, 'DEVINIVIIV', 16, 25, 10, 'H2-Kb', 40, 35.0, 'DVINIVIIV', 'DEVINIVIIV', 0.000353, 35.0], [1, 'EALPHIIDE', 9, 17, 9, 'H2-Db', 9, 31.0, 'EALPHIIDE', 'EALPHIIDE', 0.000349, 31.0], [1, 'EALPHIIDE', 9, 17, 9, 'H2-Kb', 9, 30.0, 'EALPHIIDE', 'EALPHIIDE', 0.000527, 30.0], [1, 'EALPHIIDEV', 9, 18, 10, 'H2-Db', 33, 3.6, 'EALPHIDEV', '